<a href="https://colab.research.google.com/github/linqus/audio-transformers/blob/main/notebooks/unit2/audio_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00


In [4]:
!huggingface-cli download PolyAI/minds14 --repo-type dataset --revision refs/convert/parquet --local-dir . --local-dir-use-symlinks False  --include 'en-AU/*'

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
Fetching 2 files:   0% 0/2 [00:00<?, ?it/s]downloading https://huggingface.co/datasets/PolyAI/minds14/resolve/706303078a01fcc31a36c2fc04fda5de3bdd70cf/en-AU/train/index.duckdb to /root/.cache/huggingface/hub/tmplpm_etcv
downloading https://huggingface.co/datasets/PolyAI/minds14/resolve/706303078a01fcc31a36c2fc04fda5de3bdd70cf/en-AU/train/0000.parquet to /root/.cache/huggingface/hub/tmpky1y8eca

0000.parquet:   0% 0.00/37.4M [00:00<?, ?B/s]

index.duckdb:   0% 0.00/33.6M [00:00<?, ?B/s]
0000.parquet:  28% 10.5M/37.4M [00:00<00:02, 12.6MB/s]

index.duckdb:  31% 10.5M/33.6M [00:00<00:01, 12.6MB/s]
0000.parquet:  56% 21.0M/37.4M [00:01<00:00, 22.9MB/s]

index.duckdb:  62% 21.0M/33.6M [00:01<00:00, 22.9MB/s]
0000.parquet:  84% 31.5M/37.4M [00:01<00:00, 28.0MB/s]

index.duckdb: 100% 33.6M/33.6M [00:01<00:00, 24.2MB/s]

000

In [5]:
from datasets import load_dataset
from datasets import Audio

#minds = load_dataset("PolyAI/minds14", name="en-AU", split="train")
# load from local cache
minds = load_dataset('./en-AU', split='train')
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:

from transformers import pipeline

classifier = pipeline(
    "audio-classification",
    model="anton-l/xtreme_s_xlsr_300m_minds14",
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at anton-l/xtreme_s_xlsr_300m_minds14 were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at anton-l/xtreme_s_xlsr_300m_minds14 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

In [7]:
example = minds[0]

In [8]:
classifier(example["audio"]["array"])

[{'score': 0.962530791759491, 'label': 'pay_bill'},
 {'score': 0.028672993183135986, 'label': 'freeze'},
 {'score': 0.0033498124685138464, 'label': 'card_issues'},
 {'score': 0.0020058127120137215, 'label': 'abroad'},
 {'score': 0.0008484353311359882, 'label': 'high_value_payment'}]

In [9]:
id2label = minds.features["intent_class"].int2str
id2label(example["intent_class"])

'pay_bill'